In [3]:
#This notebook needs to be run every other day!  (Every day is fine, too.)
#This will webscrape the last 3 days worth of weather data from weather.gov

# Currently working on setting it to automatically run every day, 86400 seconds

In [3]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [71]:
URL = 'https://forecast.weather.gov/data/obhistory/KBNA.html'

In [73]:
response = requests.get(URL)

In [74]:
soup = BS(response.text)

In [77]:
report_day = soup.findAll('meta', attrs={'name' : 'DC.date.created'})[0]['content']
# the format looks like: report_day = '2025-04-01T05:22:13+00:00'
report_day

'2025-03-30T13:38:30+00:00'

In [79]:
len(soup.find('tbody').findAll('tr')[0].findAll('td'))

18

In [81]:
df = pd.DataFrame(columns = ['year','month','day_of_month','time','temp','precip_1_h','precip_3_h','precip_6_h'])

In [83]:
i = 0
for row in soup.find('tbody').findAll('tr') :
    data = row.findAll('td')
    df.loc[i,'year'] = report_day[0:4]
    df.loc[i,'month'] = report_day[5:7]
    df.loc[i,'day_of_month'] = data[0].text
    df.loc[i,'time'] = data[1].text
    df.loc[i,'temp'] = data[6].text
    df.loc[i,'precip_1_h'] = data[15].text
    df.loc[i,'precip_3_h'] = data[16].text
    df.loc[i,'precip_6_h'] = data[17].text
    i += 1

In [85]:
df = df.astype({'year':'int', 'month':'int', 'day_of_month': 'int'})
df

,year,month,day_of_month,time,temp,precip_1_h,precip_3_h,precip_6_h
0,2025,3,30,06:53,66.9,,,0.02
1,2025,3,30,05:53,64.9,,,
2,2025,3,30,04:53,64.9,,,
3,2025,3,30,03:53,64.9,,0.02,
4,2025,3,30,02:53,64.9,0.02,,
...,...,...,...,...,...,...,...,...
64,2025,3,27,12:53,71.6,,,
65,2025,3,27,11:53,68,,,
66,2025,3,27,10:53,64.4,,,
67,2025,3,27,09:53,59,,,


In [87]:
df.dtypes

year             int32
month            int32
day_of_month     int32
time            object
temp            object
precip_1_h      object
precip_3_h      object
precip_6_h      object
dtype: object

In [89]:
day_part = int(report_day[8:10])
day_part

30

In [91]:
#but now I need to fix the month and year rollovers, for when the report date will show the wrong (later) month or year!
for index, row in df.iterrows() :
    if day_part < row['day_of_month'] :
    	df.loc[index,'month'] -= 1		# for example, if report day is April 1, but if the site's day is listed as 30, that needs to be adjusted to MARCH 30.
    	if row['month'] < 1 :	                    # handle the case for January 1 & 2 rollovers!  I won't use it in this project though.
    		df.loc[index,'month'] = 12
    		df.loc[index,'year'] -= 1

In [93]:
df

,year,month,day_of_month,time,temp,precip_1_h,precip_3_h,precip_6_h
0,2025,3,30,06:53,66.9,,,0.02
1,2025,3,30,05:53,64.9,,,
2,2025,3,30,04:53,64.9,,,
3,2025,3,30,03:53,64.9,,0.02,
4,2025,3,30,02:53,64.9,0.02,,
...,...,...,...,...,...,...,...,...
64,2025,3,27,12:53,71.6,,,
65,2025,3,27,11:53,68,,,
66,2025,3,27,10:53,64.4,,,
67,2025,3,27,09:53,59,,,


In [95]:
report_day

'2025-03-30T13:38:30+00:00'

In [97]:
fname = report_day.replace(':','-')
fname

'2025-03-30T13-38-30+00-00'

In [99]:
df.to_csv('../data/nws_past_3/' + fname + '.csv', index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time

print('Starting imports')

URL = 'https://forecast.weather.gov/data/obhistory/KBNA.html'

# t = 86400
t = 5

print('Starting loop')
print('')

while True:
    t -= 1
    print(f"\rSeconds left until next scrape: {t}", end="")
    if t == 0 :
        try :
            response = requests.get(URL)
            if(response.status_code == 200) :
                soup = BS(response.text)
                report_day = soup.findAll('meta', attrs={'name' : 'DC.date.created'})[0]['content']
                # the format looks like: report_day = '2025-04-01T05:22:13+00:00'
                df = pd.DataFrame(columns = ['year','month','day_of_month','time','temp','precip_1_h','precip_3_h','precip_6_h'])
                i = 0
                for row in soup.find('tbody').findAll('tr') :
                    data = row.findAll('td')
                    df.loc[i,'year'] = report_day[0:4]
                    df.loc[i,'month'] = report_day[5:7]
                    df.loc[i,'day_of_month'] = data[0].text
                    df.loc[i,'time'] = data[1].text
                    df.loc[i,'temp'] = data[6].text
                    df.loc[i,'precip_1_h'] = data[15].text
                    df.loc[i,'precip_3_h'] = data[16].text
                    df.loc[i,'precip_6_h'] = data[17].text
                    i += 1
                df = df.astype({'year':'int', 'month':'int', 'day_of_month': 'int'})
                day_part = int(report_day[8:10])
                #but now I need to fix the month and year rollovers, for when the report date will show the wrong (later) month or year!
                for index, row in df.iterrows() :
                    if day_part < row['day_of_month'] :
                    	df.loc[index,'month'] -= 1		# for example, if report day is April 1, but if the site's day is listed as 30, that needs to be adjusted to MARCH 30.
                    	if row['month'] < 1 :	                    # handle the case for January 1 & 2 rollovers!  I won't use it in this project though.
                    		df.loc[index,'month'] = 12
                    		df.loc[index,'year'] -= 1
                fname = report_day.replace(':','-')
                df.to_csv('../data/nws_past_3/' + fname + '.csv', index=False)
                print('')
                print('Saved as ' + fname)
                t = 86400
            else :
                print('')
                print('Error!  Response : ' + str(response))
                print('Trying again in 60 seconds.')
                t = 60
        except:
            print('')
            print('Something went wrong, trying again in 60 seconds.')
            print('')
            t = 60
    time.sleep(1)

Starting imports
Starting loop

Seconds left until next scrape: 0
Saved as 2025-04-01T13-58-00+00-00
Seconds left until next scrape: 86356